# STA 141B Data & Web Technologies for Data Analysis

### Lecture 11, 2/14/24, Scraping

### Today's topics
- Scraping with Java Script
- GraphQL
    
### Ressources
- Mitchell: Scraping with Python, Chapters 9 and 10
- [GraphQL](https://www.mobilelive.ca/blog/graphql-vs-rest-what-you-didnt-know) (Attention: This is infotainment!)

### Scraping from `ratemyprofessors.com`

We are interested in retrieving information from the webpage [`ratemyprofessors.com`](https://www.ratemyprofessors.com/). By navigating with our browser, we find that all professors at UCD can be retrieved as follows. 

In [ ]:
import requests

In [ ]:
endpoint = 'https://www.ratemyprofessors.com/search/professors/1073?'
params = {'q':'*'}

In [ ]:
result=requests.get(endpoint, params)
result.raise_for_status

In [ ]:
import lxml
from bs4 import BeautifulSoup

In [ ]:
html = BeautifulSoup(result.text,'lxml')
print(html.prettify()) 

In [ ]:
result.text.find("Lynn")

In [ ]:
result.text.find("Stylianos")

As we have already seen in the wikipedia example, the website (html) rendered by the browser does not coincide with the html returned by the request. Apparently, some information is fetched while the *browser* executed JS. 

The running of scripts is a client-side operation run in the browser itself, rather
than on a web server. 

JavaScript is, by far, the most common and most well-supported client-side scripting
language on the Web today. It can be used to collect information for user tracking,
submit forms without reloading the page, embed multimedia, and even power entire
online games. Even deceptively simple-looking pages can often contain multiple
pieces of JavaScript. You can find it embedded between `<script>` tags in the page’s
source code.

Since we are interested in the rendered html displayed by the browser, we have to artificialy render it first, then return the rendered html as a string. This can be achieved with `Selenium`. 

Selenium is a powerful web scraping tool developed originally for website testing.
These days it’s also used when the accurate portrayal of websites—as they appear in a
browser—is required. Selenium works by automating browsers to load the website,
retrieve the required data, and even take screenshots or assert that certain actions
happen on the website.

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
url = result.url
url

In [ ]:
# driver.get(url)

We have already seen that it takes a while to load the page in the browser. We don't have time for this. 

In [ ]:
from selenium.common.exceptions import TimeoutException
driver.set_page_load_timeout(20) # twenty seconds should be enough

try:
    driver.get(url)
except TimeoutException:
    driver.execute_script("window.stop();")

Other professors are not displayed. For that, we have to hit the button `show more`, or, better, specify that we are only interested in stats professors. 

How do we navigate on this page? First, we need to get rid of the cookies banner. Using developer tools, we can inspect find the 'close' button for the cookies banner: 

    "/html/body/div[5]/div/div/button"

See the [docs](https://www.selenium.dev/selenium/docs/api/py/index.html)!

In [ ]:
button=driver.find_element("xpath", "/html/body/div[5]/div/div/button")
button.click()

In [ ]:
button=driver.find_element("xpath", '//*[@id="bx-close-inside-1177612"]')
button.click()

Next, we should select the stats professors. To do so, we need to access the dropdown menu. From the developer tools, we find that its coded as `div` element, so we cannot use the implemented `select` method to access the dropdown. 

First, we need to find which `div` actually opens the dropdown. 

In [ ]:
driver.set_page_load_timeout(4) # twenty seconds should be enough

try:
    driver.find_element("xpath", '//div[@class=" css-1l6bn5c-control"]').click()
except TimeoutException:
    driver.execute_script("window.stop();")

Lets see how the html looks like in the dropdown menu. 

In [ ]:
import lxml.html as lx
from lxml import etree

html = lx.fromstring(driver.page_source)

In [ ]:
driver.page_source.find("Statistics")

In [ ]:
html.xpath('//div[contains(text(),"Statistics")]/@*')

In [ ]:
html.xpath('name(//div[contains(text(),"Statistics")]/@*[2])')

In [ ]:
dropdown = html.xpath('//div[@id="react-select-3-option-86"]')[0]

In [ ]:
dropdown

In [ ]:
print(BeautifulSoup(etree.tostring(dropdown),'lxml').prettify())

We ought to select the element with `id="react-select-3-option-86"`. 

In [ ]:
driver.find_element("xpath", '//div[@id="react-select-3-option-86"]').click()

We learn that there are 102 professors in the Statistics department, but only 8 are shown. Further investigation shows that we might use the class attribute that contains `Pagination button`. 

In [ ]:
button=driver.find_element("xpath", "//button[contains(@class, 'PaginationButton')]")
button.click()

In [ ]:
import time

In [ ]:
while True: 
    try: 
        time.sleep(0.2)
        button=driver.find_element("xpath", "//button[contains(@class, 'PaginationButton')]")
        button.click()
    except: 
        break

In [ ]:
html = lx.fromstring(driver.page_source)

We don't need the browser anymore. We can close it. 

In [ ]:
driver.quit()

In [ ]:
html.xpath('//a/@href')

Since we do not need visual confimation of what the browser does, we can run it in headless mode as well next time. 

In [ ]:
#chrome_options.add_argument("--headless")
#browser = webdriver.Chrome('./chromedriver', options=chrome_options)

Lets retrieve name and link for now. Any further analysis can be performed similar to our previous case studies. 

In [ ]:
links = html.xpath('//a[@class = "TeacherCard__StyledTeacherCard-syjs0d-0 dLJIlx"]/@href')
links[1:10]

In [ ]:
names = html.xpath('//div[@class = "CardName__StyledCardName-sc-1gyrgim-0 cJdVEK"]')
names = [name.text for name in names]
names[1:10]

In [ ]:
import pandas as pd

df=pd.DataFrame({'name': names, 'link': links})
df

So far so good. Next, we will see how these steps could have been achieved somewhat easier. 

We have seen that the html was rendered after some JS code has been executed. However, the information we retrieved must have been retrieved by querying some data base. To see which data base was queried using which script, we can use the performance tab in the developer tools. 

As it turns out, the information is fetched via *GraphQL*. GraphQL is an API as we have seen them before, but its not a REST API. Facebook developed it as an internal technology for their versatile applications, and later, publicly released it as open-source. Since then, the software development community has utilized it as one of the favourite technology stacks for developing web services.

As a query language, GraphQL defines specifications of how a client application can request the needed data from a remote server. As a result, the server application returns a response to the requested client query. The exciting thing to notice here is that the client application can also query exactly what it needs, without relying on the server-side application to define a query. 

GraphQL has become fairly common. Its adavantage is that due to specific queries, it avoids some problems of REST APIs, namely 
 - Multiple roundtrips with REST
 - Over-fetching and Under-fetching Problems with REST

Lets see how the GraphQL request is made. 

In [ ]:
endpoint = 'https://www.ratemyprofessors.com/graphql'
headers = {
    "Authorization": "Basic dGVzdDp0ZXN0", 
}

In [ ]:
# first query
data = {
    "query":"query TeacherSearchResultsPageQuery(\n  $query: TeacherSearchQuery!\n  $schoolID: ID\n) {\n  search: newSearch {\n    ...TeacherSearchPagination_search_1ZLmLD\n  }\n  school: node(id: $schoolID) {\n    __typename\n    ... on School {\n      name\n    }\n    id\n  }\n}\n\nfragment TeacherSearchPagination_search_1ZLmLD on newSearch {\n  teachers(query: $query, first: 8, after: \"\") {\n    didFallback\n    edges {\n      cursor\n      node {\n        ...TeacherCard_teacher\n        id\n        __typename\n      }\n    }\n    pageInfo {\n      hasNextPage\n      endCursor\n    }\n    resultCount\n    filters {\n      field\n      options {\n        value\n        id\n      }\n    }\n  }\n}\n\nfragment TeacherCard_teacher on Teacher {\n  id\n  legacyId\n  avgRating\n  numRatings\n  ...CardFeedback_teacher\n  ...CardSchool_teacher\n  ...CardName_teacher\n  ...TeacherBookmark_teacher\n}\n\nfragment CardFeedback_teacher on Teacher {\n  wouldTakeAgainPercent\n  avgDifficulty\n}\n\nfragment CardSchool_teacher on Teacher {\n  department\n  school {\n    name\n    id\n  }\n}\n\nfragment CardName_teacher on Teacher {\n  firstName\n  lastName\n}\n\nfragment TeacherBookmark_teacher on Teacher {\n  id\n  isSaved\n}\n",
    "variables":{
        "query":{
            "text":"",
            "schoolID":"U2Nob29sLTEwNzM=",
            "fallback":True,
            "departmentID":"RGVwYXJ0bWVudC0xNDA=", 
        },
        "schoolID":"U2Nob29sLTEwNzM="
    }
} 

In [ ]:
response = requests.post(endpoint, headers = headers, json=data)
response.raise_for_status()
result = response.json()
result

In [ ]:
def fetch_info(dic): 
    name = dic['node']['firstName'] + " " + dic['node']['lastName']
    lid = "/professor?tid=" + str(dic['node']['legacyId'])
    return name, lid
    
prof_list = result['data']['search']['teachers']['edges']
    
[fetch_info(prof) for prof in prof_list]

Using developer tools, we find the the subsequent requests can be done using a different data layout. Watch out, the `query` value has changed! 

In [ ]:
cursor = result['data']['search']['teachers']['pageInfo']['endCursor']
cursor

In [ ]:
def new_data(cursor):
    data = {
        "query":"query TeacherSearchPaginationQuery(\n  $count: Int!\n  $cursor: String\n  $query: TeacherSearchQuery!\n) {\n  search: newSearch {\n    ...TeacherSearchPagination_search_1jWD3d\n  }\n}\n\nfragment TeacherSearchPagination_search_1jWD3d on newSearch {\n  teachers(query: $query, first: $count, after: $cursor) {\n    didFallback\n    edges {\n      cursor\n      node {\n        ...TeacherCard_teacher\n        id\n        __typename\n      }\n    }\n    pageInfo {\n      hasNextPage\n      endCursor\n    }\n    resultCount\n    filters {\n      field\n      options {\n        value\n        id\n      }\n    }\n  }\n}\n\nfragment TeacherCard_teacher on Teacher {\n  id\n  legacyId\n  avgRating\n  numRatings\n  ...CardFeedback_teacher\n  ...CardSchool_teacher\n  ...CardName_teacher\n  ...TeacherBookmark_teacher\n}\n\nfragment CardFeedback_teacher on Teacher {\n  wouldTakeAgainPercent\n  avgDifficulty\n}\n\nfragment CardSchool_teacher on Teacher {\n  department\n  school {\n    name\n    id\n  }\n}\n\nfragment CardName_teacher on Teacher {\n  firstName\n  lastName\n}\n\nfragment TeacherBookmark_teacher on Teacher {\n  id\n  isSaved\n}\n",
        "variables":{
            "count":8,
            "cursor": cursor, 
            "query":{
                "text":"",
                "schoolID":"U2Nob29sLTEwNzM=",
                "fallback":True,
                "departmentID":"RGVwYXJ0bWVudC0xNDA=", 
            }
        }
    } 
    return data
data = new_data(cursor)

In [ ]:
response = requests.post(endpoint, headers = headers, json=data)
response.raise_for_status()
result = response.json()
result

In [ ]:
prof_list = result['data']['search']['teachers']['edges']
[fetch_info(prof) for prof in prof_list]

In [ ]:
cursor = result['data']['search']['teachers']['pageInfo']['endCursor']
cursor

In [ ]:
flag = result['data']['search']['teachers']['pageInfo']['hasNextPage']
flag

Lets formalize this. 

In [ ]:
def fetch_profs(): 
    endpoint = 'https://www.ratemyprofessors.com/graphql'
    headers = {
        "Authorization": "Basic dGVzdDp0ZXN0", 
    }
    
    # first query
    data = {
        "query":"query TeacherSearchResultsPageQuery(\n  $query: TeacherSearchQuery!\n  $schoolID: ID\n) {\n  search: newSearch {\n    ...TeacherSearchPagination_search_1ZLmLD\n  }\n  school: node(id: $schoolID) {\n    __typename\n    ... on School {\n      name\n    }\n    id\n  }\n}\n\nfragment TeacherSearchPagination_search_1ZLmLD on newSearch {\n  teachers(query: $query, first: 8, after: \"\") {\n    didFallback\n    edges {\n      cursor\n      node {\n        ...TeacherCard_teacher\n        id\n        __typename\n      }\n    }\n    pageInfo {\n      hasNextPage\n      endCursor\n    }\n    resultCount\n    filters {\n      field\n      options {\n        value\n        id\n      }\n    }\n  }\n}\n\nfragment TeacherCard_teacher on Teacher {\n  id\n  legacyId\n  avgRating\n  numRatings\n  ...CardFeedback_teacher\n  ...CardSchool_teacher\n  ...CardName_teacher\n  ...TeacherBookmark_teacher\n}\n\nfragment CardFeedback_teacher on Teacher {\n  wouldTakeAgainPercent\n  avgDifficulty\n}\n\nfragment CardSchool_teacher on Teacher {\n  department\n  school {\n    name\n    id\n  }\n}\n\nfragment CardName_teacher on Teacher {\n  firstName\n  lastName\n}\n\nfragment TeacherBookmark_teacher on Teacher {\n  id\n  isSaved\n}\n",
        "variables":{
            "query":{
                "text":"",
                "schoolID":"U2Nob29sLTEwNzM=",
                "fallback":True,
                "departmentID":"RGVwYXJ0bWVudC0xNDA=", 
            },
            "schoolID":"U2Nob29sLTEwNzM="
        }
    } 
    
    response = requests.post(endpoint, headers = headers, json=data)
    result = response.json()
    
    prof_list = result['data']['search']['teachers']['edges']
    df = [fetch_info(prof) for prof in prof_list]
    
    cursor = result['data']['search']['teachers']['pageInfo']['endCursor']
    
    flag = True
    while flag: 
        data = new_data(cursor)
        response = requests.post(endpoint, headers = headers, json=data)
        result = response.json()
            
        prof_list = result['data']['search']['teachers']['edges']
        df.extend([fetch_info(prof) for prof in prof_list])
        cursor = result['data']['search']['teachers']['pageInfo']['endCursor']

        flag = result['data']['search']['teachers']['pageInfo']['hasNextPage']
        
    return df

In [ ]:
df = fetch_profs()

In [ ]:
pd.DataFrame(df)

### Summary 

- `Selenium` is very useful to remote-control a browser
- Internally, information is usually handled via APIs anyway